In [ ]:
from helper import *

import pandas as pd
import numpy as np
import os
import csv
import sys
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 8]
# use LaTeX fonts in the plot
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
%matplotlib inline

In [ ]:
OUTPUT = "single"
EVAL_DIR = "/home/felix/todo/algohol/single"
MLP_METHODS = ["kmeans", "gonzalez", "merge"]
MLP_LEVELS = [[int(2 ** i)] for i in np.arange(9.0, 12.5, 1.0)]
AREAS = ["baden-wuerttemberg"]
print(MLP_METHODS, "with", MLP_LEVELS)

In [ ]:
# OUTPUT = "bicycle"
# EVAL_DIR = "/home/felix/todo/algohol/bicycle"
# MLP_METHODS = ["merge"]
# MLP_LEVELS = [[int(2 ** i)] for i in np.arange(13.0, 15.5, 1.0)]
# AREAS = ["baden-wuerttemberg"]
# SKIP_PCH = True
# print(MLP_METHODS, "with", MLP_LEVELS)

In [ ]:
with open(EVAL_DIR + "/log.json") as file:
    tmp = file.readlines()
    log = pd.read_json(" ".join(tmp), orient='index')

In [ ]:
# create command string to not match against list
log['command_string'] = log['command'].agg(lambda x: ' '.join(map(str, x)))

# plots

In [ ]:
fig, ax = plt.subplots()

for method in MLP_METHODS:
    x = list()
    y = list()
    for level in MLP_LEVELS:
        lines = log[(log['command_string'].str.contains("prp_pre")) & (log['command_string'].str.contains(method + "-" + "_".join(map(str, level)) + ".bin"))]
        if lines.shape[0] == 0:
            continue
        # take last line of execution
        line = lines.iloc[-1]
        runtime = line['time'].replace(" seconds", "")
        x.append(level)
        y.append(float(runtime))
    ax.plot(x, y, color=plot_get(method))
    ax.scatter(x, y, color=plot_get(method), label=mlp_title(method))
    
ax.set_xlabel("Partitions")
ax.set_ylabel("Preprocessing [sec]")
# ax.set_yscale('log')
ax.legend(loc='upper right')
fig.savefig(OUTPUT+ "-pre-runtime.pgf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots()
# fig, ax = plt.subplots(figsize=(4, 3))
table = list()

for area in AREAS:
    for mlp in MLP_METHODS:
        x = list()
        y = list()
        for level in MLP_LEVELS:
            lines = log[(log['command_string'].str.contains("prp_pre")) & (log['command_string'].str.contains(mlp + "-" + "_".join(map(str, level)) + ".bin"))]
            if lines.shape[0] == 0:
                continue
            # take last line of execution
            line = lines.iloc[-1]
            runtime = line['time'].replace(" seconds", "")
            df_pch_csv = pd.read_csv(EVAL_DIR + "/" + area + "-" + mlp + "-" + "_".join(map(str, level)) + ".csv")
            contracted_nodes = sum(df_pch_csv.highest_diff.value_counts().loc[1:])
            percentage = 1 - (contracted_nodes / len(df_pch_csv))
            x.append(percentage * 100.0)
            y.append(float(runtime))
            current = {"MLP": mlp_title(mlp), "_".join(map(str, level)) + "-Percentage": "{:0.2f} %".format(percentage * 100.0), "_".join(map(str, level)) + "-pre [min]": int(sec_to_min(float(runtime)))}
            table.append(current)
        ax.plot(x, y, color=plot_get(mlp))
        ax.scatter(x, y, color=plot_get(mlp), label=mlp_title(mlp))
    
ax.set_xlabel("Contracted nodes [\%]")
ax.set_ylabel("Preprocessing [sec]")
# ax.set_yscale('log')
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax.legend(loc='upper left')
fig.savefig(OUTPUT + "-pre-percentage-runtime.pgf", bbox_inches="tight")

In [ ]:
df_table = pd.DataFrame(table)
assert((df_table.groupby(["MLP"]).count().all() == 1).iloc[0])
df_table = df_table.groupby(["MLP"], sort = False).first()
latex = df_table.to_latex(float_format="{0:.0f}".format)
df_table

In [ ]:
# fixup ugly latex code to have single line header
latex_list = latex.splitlines()

columns = latex_list[2].split("&")
indices = latex_list[3].split("&")

latex_list[2] = ""
for level in MLP_LEVELS:
    latex_list[2] += "& \multicolumn{2}{c|}{" + "_".join(map(str, level)) + "} "
latex_list[2] += " \\\\"
# remove last pipe from "c"
latex_list[2] = "c".join(latex_list[2].rsplit("c|", 1))

latex_list[3] = "&".join(indices[:1] + columns[1:])

for level in MLP_LEVELS:
    latex_list[3] = latex_list[3].replace("_".join(map(str, level)) + "-Percentage", "contract").replace("_".join(map(str, level)) + "-pre [min]", "[min]")

latex_list[0] = latex_list[0].replace('ll', 'l|l', 1)
latex_list[0] = latex_list[0].replace('lr', 'rr|', len(MLP_LEVELS) - 1)
latex_list[0] = latex_list[0].replace('lr', 'rr', 1)

# latex_list.insert(len(latex_list)-6, '\midrule')
# latex_list.insert(len(latex_list)-5, '\midrule')

latex_new = '\n'.join(latex_list)
with open(OUTPUT + "-pre.tex", "w") as latex_file:
    latex_file.writelines(latex_new)